#### This project will explore, segment, and cluster the neighborhoods in the city of Toronto

##### Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import geocoder
from bs4 import BeautifulSoup
import ssl
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

ssl._create_default_https_context = ssl._create_unverified_context

##### Getting the contents of the website with 'requests' library

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
response

<Response [200]>

##### Scraping the contents from html using BeautifulSoup and building an initial Pandas dataframe

In [3]:
# instantiating the soup object with response text, and html.parser option
soup = BeautifulSoup(response.text, "html.parser")

# parsing the table part of the response by looking at wikitable sortable class-type
postal_table = soup.find(class_="wikitable sortable")

# building the initial dataframe from table's contents 
table_rows = postal_table.find_all('tr')
row_values = []
for tr in table_rows:
    td = tr.find_all('td')
    row_text = [tr.text.strip() for tr in td if tr.text.strip()]
    if row_text:
        row_values.append(row_text)

toronto_df = pd.DataFrame(row_values, columns=["PostalCode", "Borough", "Neighborhood"])
# toronto_df.head(10)
toronto_df.shape


(288, 3)

##### Cleaning the dataframe (dropping, combining, and truncating multiple cells)

In [5]:
# ignoring cells with a Borough that is Not assigned.
borough_df = toronto_df[toronto_df.Borough != 'Not assigned']

# replacing 'Not assigned' neighborhood value with the corresponding Borough value
borough_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)


# combining neighborhoods with the same PostalCode into single row 
combined_df = borough_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index(drop=False)

#combined_df.head(10)
combined_df.shape


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


(103, 3)

##### Getting the latitude and longitude coordinates of each neighborhood to utilize the Foursquare location data

In [6]:
# geocoder seems to be very unreliable, so we can use the link to the csv file to get latitude and longitude
url = 'http://cocl.us/Geospatial_data'
lat_long_df = pd.read_csv(url)

# since the latitude-longitude table has the same data ordered as of combined_df, we can just create a new dataframe
# with the required columns
detailed_df = pd.DataFrame({'PostalCode':combined_df['PostalCode'], 
                            'Borough':combined_df['Borough'], 
                            'Neighborhood':combined_df['Neighborhood'], 
                            'Latitude':lat_long_df['Latitude'], 
                            'Longitude':lat_long_df['Longitude']})


print("Toronto has a total of {} boroughs and {} neighborhoods.".format(len(detailed_df.Borough.unique()), len(detailed_df.Neighborhood.unique())))



Toronto has a total of 11 boroughs and 103 neighborhoods.


##### Cluster analysis of the neighborhoods in Toronto 

Toronto neighborhood has a total of 11 boroughs and 100+ neighborhoods. It will become a tedious task to analyse all 11 boroughs, so we will work on boroughs that has the word 'Toronto' in them. There are totally 4 boroughs that has 'toronto' in their name: "Downtown Toronto", "Central Toronto", "West Toronto", and "East Toronto".

In [7]:
# Analysing number of postal codes in '** toronto' borough
toronto_borough = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
for tor in toronto_borough:
    print("{} has a total of {} postal codes.".format(tor, detailed_df[detailed_df['Borough'] == tor].PostalCode.count()))



Downtown Toronto has a total of 18 postal codes.
Central Toronto has a total of 9 postal codes.
West Toronto has a total of 6 postal codes.
East Toronto has a total of 5 postal codes.


In [11]:
# Creating a new dataframe for the cluster analysis of 'Toronto' Boroughs
d_t = detailed_df[detailed_df['Borough'] == 'Downtown Toronto']
c_t = detailed_df[detailed_df['Borough'] == 'Central Toronto']
w_t = detailed_df[detailed_df['Borough'] == 'West Toronto']
e_t = detailed_df[detailed_df['Borough'] == 'East Toronto']

combined = pd.concat([d_t, c_t, w_t, e_t], sort=False)
toronto_dataframe = combined.reset_index(drop=True)

# toronto_dataframe.head()
toronto_dataframe.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [9]:
# Using geopy to get the latitude and longitude values of Toronto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}. '.format(latitude, longitude))

Coordinates of Toronto are 43.653963, -79.387207. 


##### Creating a map of Toronto with all its neighborhoods superimposed on Top (filtered by Borough that has the word 'toronto')

In [12]:
# creating a map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers
for lat, lng, borough, neighborhood in zip(toronto_dataframe['Latitude'], toronto_dataframe['Longitude'], toronto_dataframe['Borough'], toronto_dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
    
map_toronto

##### Exploring first neighborhood and it's venues with Foursquare API

In [13]:
# foursquare credentials to access their APIs
client_id = 'PEQPJEZXPG4HRTFTMTZHL3P3QRZ1SV55525PX2YO22F3Q0JB'
client_secret = 'JEDPFGTRY4SISHI1S2W3RTQGIG3UUROA5OG4I3NW4UXBBLB1'
version = '20180605'

In [16]:
# first neighborhood in toronto_dataframe
toronto_dataframe.loc[0, 'Neighborhood']

'Rosedale'

In [17]:
# getting Rosedale's latitude and longitude values
rosedale_latitude = toronto_dataframe.loc[0, 'Latitude']
rosedale_longitude = toronto_dataframe.loc[0, 'Longitude']
print('Rosedale\'s latitude and longitude values are {}, {}.'.format(rosedale_latitude, rosedale_longitude))

Rosedale's latitude and longitude values are 43.6795626, -79.37752940000001.


In [19]:
# getting the top 100 venues that are in Rosedale within a radius of 500 meteres
no_of_venues = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, rosedale_latitude, rosedale_longitude, radius, no_of_venues)

response = requests.get(url).json()
response


{'meta': {'code': 200, 'requestId': '5d2407d3a30619002ccaf863'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [20]:
# function to extract the venue's category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
# cleaning the response json and structuring it into pandas dataframe
venues = response['response']['groups'][0]['items']

# flatten JSON
nearby_venues = json_normalize(venues)

# filtering only needed columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


Rosedale neighborhood has 5 venues

##### Exploring all neighborhoods and it's venues with Foursquare API

Now we are going to explore all the neighborhoods of four different Boroughs (ending with 'toronto') using Foursquare APIs and above mentioned process

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, lat, lng, radius, no_of_venues)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)

In [44]:
# code to run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=toronto_dataframe['Neighborhood'],
                                 latitudes=toronto_dataframe['Latitude'],
                                 longitudes=toronto_dataframe['Longitude'])



Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
The Beaches
The Danforth West, 

In [47]:
# checking how many venues were returned overall
print(toronto_venues.shape)
toronto_venues.head(10)

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
5,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
6,"Cabbagetown, St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
7,"Cabbagetown, St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
8,"Cabbagetown, St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
9,"Cabbagetown, St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant


In [57]:
# number of venues for each neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide, King, Richmond                                                                                      100
Berczy Park                                                                                                    55
Brockton, Exhibition Place, Parkdale Village                                                                   22
Business Reply Mail Processing Centre 969 Eastern                                                              19
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara     16
Cabbagetown, St. James Town                                                                                    46
Central Bay Street                                                                                             88
Chinatown, Grange Park, Kensington Market                                                                     100
Christie                                                                   

In [67]:
# number of unique categories of venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


##### One-hot encoding all neighborhoods based on the Venue Category


In [77]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1700, 239)

##### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [83]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head(5)

(38, 239)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.000000,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.00,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.045455,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


##### Each neighborhood with top 5 common venues

In [88]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("[--------"+hood+"--------]")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

[--------Adelaide, King, Richmond--------]
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.04


[--------Berczy Park--------]
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3                Café  0.04
4            Beer Bar  0.04


[--------Brockton, Exhibition Place, Parkdale Village--------]
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4          Bakery  0.05


[--------Business Reply Mail Processing Centre 969 Eastern--------]
              venue  freq
0       Yoga Studio  0.05
1     Auto Workshop  0.05
2              Park  0.05
3        Comic Shop  0.05
4  Recording Studio  0.05


[--------CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara--------]
              venue  

##### Creating a dataframe that has top 10 venues for each neighborhood

In [120]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
# create a dataframe to display top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd', 'th']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
         columns.append('{}th Most Common Venue'.format(ind+1))
        

# create the new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

    
neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,American Restaurant,Steakhouse,Hotel,Restaurant,Cosmetics Shop,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Farmers Market,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Steakhouse,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Pet Store,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Plane,Coffee Shop,Boat or Ferry,Sculpture Garden,Boutique,Bar,Airport Gate


##### Run K-Means to cluster the neighborhoods

In [178]:
# set the number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#checking cluster lables generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

##### New dataframe that includes the clusters


In [179]:
# add cluster labels
neighborhood_venues_sorted
toronto_merged = toronto_dataframe

# merge toronto_grouped with toronto_dataframe to add lat/long for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,Building,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Breakfast Spot,Sandwich Place
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Pub,Burger Joint,Hotel,Café
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Theater,Lingerie Store,Restaurant,Ramen Restaurant


##### Visualizing the clusters

In [180]:
# creating map
cluster_map = folium.Map(location=[latitude, longitude], zoom_starts=17)

# setting color schemes
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=9,
    popup=label,
    colors=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=2.0).add_to(cluster_map)
    

    
cluster_map